In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [7]:
import os
RAW_DIR = '../data/raw/'
PROCESSED_DIR = '../data/processed/'

DATASET_NAME = 'ucidata-zachary'
DS_INPUT = os.path.join(RAW_DIR, DATASET_NAME)
DS_OUTPUT = os.path.join(PROCESSED_DIR, DATASET_NAME)

In [5]:
spark = (SparkSession.builder
         .appName(f'{DATASET_NAME}_preprocess')
         .config('spark.sql.legacy.timeParserPolicy', 'LEGACY')
         .getOrCreate())

21/12/30 14:44:49 WARN Utils: Your hostname, megatron resolves to a loopback address: 127.0.1.1; using 192.168.1.89 instead (on interface enp7s0)
21/12/30 14:44:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/30 14:44:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [19]:
df = (
    spark.read.csv(os.path.join(DS_INPUT, 'out.ucidata-zachary'), sep=' ')
        .withColumn('index', F.monotonically_increasing_id())
        .filter(F.col('index') > 1)
        .withColumnRenamed('_c0', 'src')
        .withColumnRenamed('_c1', 'dst')
)
df.head(5)

[Row(src='1', dst='2', _c2=None, index=2),
 Row(src='1', dst='3', _c2=None, index=3),
 Row(src='2', dst='3', _c2=None, index=4),
 Row(src='1', dst='4', _c2=None, index=5),
 Row(src='2', dst='4', _c2=None, index=6)]

In [20]:
df_nodes = (
    df.select(F.col('src').alias('id'))
        .union(df.select(F.col('dst').alias('id')))
        .distinct()
)
df_nodes.head(5)

[Row(id='7'), Row(id='15'), Row(id='29'), Row(id='3'), Row(id='30')]

In [21]:
df_edges = (
    df.select(F.col('src'), F.col('dst'), F.lit(1).alias('undirected'))
        .distinct()
)
df_edges.head(5)

[Row(src='24', dst='28', undirected=1),
 Row(src='30', dst='34', undirected=1),
 Row(src='25', dst='26', undirected=1),
 Row(src='31', dst='34', undirected=1),
 Row(src='10', dst='34', undirected=1)]

In [22]:
df_nodes.write.parquet(os.path.join(DS_OUTPUT, 'nodes_default'), mode='overwrite')
df_edges.write.parquet(os.path.join(DS_OUTPUT, 'edges_default'), mode='overwrite')